In [1]:
import pandas as pd
import numpy as np
import sklearn
from keras.models import Sequential
from keras.layers import Dense, Dropout, Masking, Embedding
from matplotlib import pyplot as plt
from ipywidgets import interact
import pickle
plt.rcParams['figure.dpi'] = 200

In [2]:
file_headers = ["/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/module1_fullext1",
                "/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/module1_fullext2",
                "/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/module1_fullext3",
                "/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/module1_fullext4",
                "/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/module2_fullext1",
                "/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/module2_fullext2",
                "/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/module2_fullext3",
                "/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/module2_fullext4",
                "/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/s_curve1",
                "/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/s_curve2"]

CSV_SFX = ".csv"
MARKERS_SFX = "_markers"
POLY_SFX = "_poly"
M10Y = "M10Y"

In [3]:
# distance goes from 22-38. 16 cm of extension at least which is pretty good.

In [4]:
poly_df_list = []
marker_df_list = []
data_df_list = []

for header in file_headers:
    data_file = header + CSV_SFX
    marker_file = header + MARKERS_SFX + CSV_SFX
    poly_file = header + POLY_SFX + CSV_SFX
    
    data_df = pd.read_csv(data_file)
    marker_df = pd.read_csv(marker_file)
    poly_df = pd.read_csv(poly_file)
    
    zero_indices = list(marker_df[marker_df[M10Y] == 0].index)
    
    data_df = data_df.drop(data_df.index[zero_indices]).reset_index()
    marker_df = marker_df.drop(marker_df.index[zero_indices])
    poly_df = poly_df.drop(poly_df.index[zero_indices])
    
    data_df_list.append(data_df)
    marker_df_list.append(marker_df)
    poly_df_list.append(poly_df)

total_data_df = pd.concat(data_df_list).reset_index()
total_marker_df = pd.concat(marker_df_list).reset_index()
total_poly_df = pd.concat(poly_df_list).reset_index()


training_data_list = [total_data_df, total_marker_df, total_poly_df]
all_df = pd.concat(training_data_list, axis=1)

y_labels = ["M1-PL", "M1-PR", "M2-PL", "M2-PR"]
# NOTE: Uncomment if using polynomial
# y_labels = ["a0", "a1", "a2", "a3", "a4", "a5", "d"]

# NOTE: Uncomment if using markers
# y_labels = ["M1X", "M1Y"]
x_labels = ["M1X", "M1Y", "M2X", "M2Y", "M3X", "M3Y",
            "M4X", "M4Y", "M5X", "M5Y", "M6X", "M6Y",
            "M7X", "M7Y", "M8X", "M8Y", "M9X", "M9Y",
            "M10X", "M10Y"]

MX_min = -15
MX_max = 15
MY_min = 0
MY_max = 40

P_min = 95
P_max = 121

def normalize_columns(df, col_list):
    for c in col_list:
        if "X" in c:
            min_val = MX_min
            max_val = MX_max
        elif "Y" in c:
            min_val = MY_min
            max_val = MY_max
        elif "P" in c:
            min_val = P_min
            max_val = P_max
        #print("column: " + str(c) + ", min: " + str(min_val) + ", max: " + str(max_val))
        df[c] = (df[c] - min_val) / (max_val - min_val)
        
normalize_columns(all_df, x_labels)
normalize_columns(all_df, y_labels)

In [5]:
# Prepare data for DNN
# shuffle the rows
#train_df = all_df.sample(frac=0.8, random_state=25)
#test_df = all_df.drop(train_df.index)

x_dim = len(x_labels)
y_dim = len(y_labels)

x_data = all_df[x_labels]
y_data = all_df[y_labels]

x_data_shuffled, y_data_shuffled = sklearn.utils.shuffle(x_data, y_data)

split = int(0.8 * len(x_data))

x_train = x_data_shuffled[:split]
y_train = y_data_shuffled[:split]

x_test = x_data_shuffled[split:]
y_test = y_data_shuffled[split:]

In [6]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(4895, 20)
(1224, 20)
(4895, 4)
(1224, 4)


# DNN

In [7]:
model = Sequential()
model.add(Dense(128, input_dim=x_dim, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(y_dim, activation='relu'))

model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               2688      
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                2080      
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 16)                5

In [8]:
history = model.fit(x_train, y_train, epochs=100, batch_size=8)

Epoch 1/100
612/612 [==============================] - 1s 453us/step - loss: 0.0614 - accuracy: 0.3128
Epoch 2/100
612/612 [==============================] - 0s 452us/step - loss: 0.0241 - accuracy: 0.5226
Epoch 3/100
612/612 [==============================] - 0s 451us/step - loss: 0.0163 - accuracy: 0.5961
Epoch 4/100
612/612 [==============================] - 0s 446us/step - loss: 0.0146 - accuracy: 0.6155
Epoch 5/100
612/612 [==============================] - 0s 447us/step - loss: 0.0134 - accuracy: 0.6161
Epoch 6/100
612/612 [==============================] - 0s 443us/step - loss: 0.0129 - accuracy: 0.5906
Epoch 7/100
612/612 [==============================] - 0s 441us/step - loss: 0.0120 - accuracy: 0.6089
Epoch 8/100
612/612 [==============================] - 0s 445us/step - loss: 0.0118 - accuracy: 0.6216
Epoch 9/100
612/612 [==============================] - 0s 467us/step - loss: 0.0109 - accuracy: 0.6238
Epoch 10/100
612/612 [==============================] - 0s 446us/step - l

612/612 [==============================] - 0s 445us/step - loss: 0.0080 - accuracy: 0.6669
Epoch 80/100
612/612 [==============================] - 0s 442us/step - loss: 0.0085 - accuracy: 0.6514
Epoch 81/100
612/612 [==============================] - 0s 446us/step - loss: 0.0085 - accuracy: 0.6535
Epoch 82/100
612/612 [==============================] - 0s 447us/step - loss: 0.0083 - accuracy: 0.6424
Epoch 83/100
612/612 [==============================] - 0s 450us/step - loss: 0.0086 - accuracy: 0.6518
Epoch 84/100
612/612 [==============================] - 0s 450us/step - loss: 0.0079 - accuracy: 0.6468
Epoch 85/100
612/612 [==============================] - 0s 444us/step - loss: 0.0080 - accuracy: 0.6564
Epoch 86/100
612/612 [==============================] - 0s 443us/step - loss: 0.0085 - accuracy: 0.6452
Epoch 87/100
612/612 [==============================] - 0s 446us/step - loss: 0.0087 - accuracy: 0.6426
Epoch 88/100
612/612 [==============================] - 0s 443us/step - loss:

In [45]:
errors = []
for i,x in x_test.iterrows():
    inp = np.array(x).reshape(1,-1)
    y = model.predict(inp)[0]
    y_true = np.array(y_test.loc[i,:])
    error = 0.0
    for idx in range(len(y)):
        error += ((y[idx] - y_true[idx]) ** 2) * (1/len(y))
    errors.append(error)
mse = sum(errors) / len(errors)
print(mse)
    

0.0008994790001281283


In [53]:
loss, accuracy = model.evaluate(x_test, y_test)
print('Loss: %.8f, Accuracy: %.2f' % (loss, (accuracy*100)))

39/39 [==============================] - 0s 368us/step - loss: 0.0029 - accuracy: 0.7010
Loss: 0.00286911, Accuracy: 70.10


In [23]:
pickle.dump(history.history["loss"], open("objects/dnn_loss.p", "wb"))

In [13]:
def rescale_output(y):
    ret = []
    for i in range(len(y)):
        max_val = y_maxes[i]
        min_val = y_mins[i]
        val = y[i]
        rescaled_val = (val * (max_val - min_val)) + min_val
        ret.append(rescaled_val)
    return np.array(ret)

In [14]:
num_tests = len(x_test)
def evaluate_poly(idx=(0,num_tests-1,1)):
    index = x_test.index[idx]
    x_select = np.array(x_test.loc[index, :]).reshape(1, -1)
    y_pred = model.predict(x_select)
    y_pred = rescale_output(y_pred[0])
    a_pred = np.flip(y_pred[:-1])
    d_pred = y_pred[-1]
    
    y_select = np.array(y_test.loc[index, :]).reshape(1, -1)
    y_select = rescale_output(y_select[0])
    a_select = np.flip(y_select[:-1])
    d_select = y_select[-1]
    
    print(d_pred, d_select)
    
    poly_pred = np.poly1d(a_pred)
    poly_select = np.poly1d(a_select)
    
    yp = np.linspace(0, d_pred)
    xp = poly_pred(yp)
    
    ys = np.linspace(0, d_select)
    xs = poly_select(ys)
    
    plt.plot(xs, ys, label="actual")
    plt.plot(xp, yp, label="predicted")
    plt.xlim([-20,20])
    plt.ylim([-1,39])
    plt.xlabel("x (cm)")
    plt.ylabel("y (cm)")
    plt.legend()
    plt.title("Predicted Shape vs. Actual Shape")
    plt.show()
    

interact(evaluate_poly)

Widget Javascript not detected.  It may not be installed or enabled properly.


<function __main__.evaluate_poly(idx=(0, 1223, 1))>

In [11]:
def rescale_input(y):
    ret = []
    for i in range(len(y)):
        max_val = P_max
        min_val = P_min
        val = y[i]
        rescaled_val = (val * (max_val - min_val)) + min_val
        ret.append(rescaled_val)
    return np.array(ret)

def rescale_output(y):
    ret = []
    for i in range(len(y)):
        if (i % 2) == 0:
            max_val = MX_max
            min_val = MX_min
        else:
            max_val = MY_max
            min_val = MY_min
        val = y[i]
        rescaled_val = (val * (max_val - min_val)) + min_val
        ret.append(rescaled_val)
    return np.array(ret)

In [54]:
num_tests = len(x_test)
def evaluate_markers(idx=(0,num_tests-1,1)):
    index = x_test.index[idx]
    x_select = np.array(x_test.loc[index, :]).reshape(1, -1)
    print(x_select.shape)
    print(rescale_input(x_select))
    y_pred = model.predict(x_select)
    y_pred = rescale_output(y_pred[0])
    
    y_select = np.array(y_test.loc[index, :]).reshape(1, -1)
    y_select = rescale_output(y_select[0])
    
    num_markers = int(len(y_pred) / 2)
    
    x_coord_select = [0]
    y_coord_select = [0]
    x_coord_pred = [0]
    y_coord_pred = [0]
    
    for i in range(num_markers):
        x_idx = i * 2
        y_idx = (i * 2) + 1
        x_coord_select.append(y_select[x_idx])
        y_coord_select.append(y_select[y_idx])
        x_coord_pred.append(y_pred[x_idx])
        y_coord_pred.append(y_pred[y_idx])
    
    plt.plot(x_coord_select, y_coord_select, "-o", label="actual")
    plt.plot(x_coord_pred, y_coord_pred, "-o", label="predicted")
    plt.xlim([-20,20])
    plt.ylim([-1,39])
    plt.xlabel("x (cm)")
    plt.ylabel("y (cm)")
    plt.legend()
    plt.title("Predicted Markers vs. Actual Markers")
    plt.show()
        
        
interact(evaluate_markers)

Widget Javascript not detected.  It may not be installed or enabled properly.


<function __main__.evaluate_markers(idx=(0, 1223, 1))>

In [149]:
model.save("./models/first_model")

INFO:tensorflow:Assets written to: ./models/first_model/assets


In [20]:
num_tests = len(x_test)
def evaluate_pressures(idx=(0,num_tests-1,1)):
    index = x_test.index[idx]
    print(x_test)
    x_select = np.array(x_test.loc[index, :]).reshape(1,-1)
    print(x_select)
interact(evaluate_pressures)

Widget Javascript not detected.  It may not be installed or enabled properly.


<function __main__.evaluate_pressures(idx=(0, 1223, 1))>

In [ ]:
# First model tried
model = Sequential()
model.add(Dense(12, input_dim=x_dim, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(y_dim, activation='linear'))
model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

In [28]:
num_tests = len(x_test)
def evaluate_pressures(idx=(0,num_tests-1,1)):
    index = x_test.index[idx]
    print(y_test)
    x_select = np.array(x_test.loc[index, :]).reshape(1, -1)
    y_pred = model.predict(x_select)
    y_pred = rescale_input(y_pred[0])
    
    y_select = np.array(y_test.loc[index, :]).reshape(1, -1)
    y_select = rescale_input(y_select[0])
    
    print(y_select)
    m1 = [0,1]
    m2 = [2,3]
    
    plt.plot(m1, y_select[:2], "-x", label="m1 left actual", color="red")
    plt.plot(m1, y_pred[:2], "-x", label="m1 right predicted", color ="blue")
    plt.plot(m2, y_select[2:], "-x", label="m2 left actual", color="red")
    plt.plot(m2, y_pred[2:], "-x", label="m2 right predicted", color="blue")
    plt.ylabel("kPa")
    plt.xlim([-1,4])
    plt.ylim([95,125])
    plt.legend()
    plt.title("Predicted Pressures vs. Actual Pressures")
    plt.show()
        
        
interact(evaluate_pressures)

Widget Javascript not detected.  It may not be installed or enabled properly.


<function __main__.evaluate_pressures(idx=(0, 1223, 1))>

In [65]:
# BAD normalize method
def normalize_columns(df, col_list):
    min_vals = []
    max_vals = []
    for c in col_list:
        min_val = df[c].min()
        max_val = df[c].max()
        min_vals.append(min_val)
        max_vals.append(max_val)
        print("column: " + str(c) + ", min: " + str(min_val) + ", max: " + str(max_val))
        df[c] = (df[c] - min_val) / (max_val - min_val)
    return min_vals, max_vals